# Основное предсказание

Целью обучения является предсказание именно обратной волатильности с последующем использованием ее в торговле.

Часть I
Собственно предсказание волатильности. Результат - минимизация ошибки предсказания.
Часть II
Расширением задачи является использование полученной волатильности в бэктестировании. Выход сети будет использоваться для дельта-хеджирования для опционной стратегии. Результат торговли должен быть оценен по коэффициенту Шарпа (или Сортино) и обучение будет вестись не для минимизации ошибки какого-либо выхода сети, а на максимизацию заданного критерия. При этом выходов сети может быть больше одного - например для задания максимального и минимального прогнозируемых значений волатильности или для задания набора пар волатильность/вероятность ее реализации
Нужно проверить (по усложнению):
перебрать разные, но общие для всех волатильностей периоды усреднения 
разные усреднения для разных волатильностей
продублировать фичи для усреднений (как минимум двух). т.е. чтобы каждая волатильность была рассчитана дважды с разным усреднением


1. делим истор. данные на train, test и eval.
2. Из train получаем средние и дисперсии, используем их чтобы отнормировать фичи и change на train, test и eval.
3. Из train получаем порог, по которому разделить change, чтобы скачков -1, 0 и +1 было поровну на train.
4. По этому же порогу разделяем change на -1, 0 и +1 на test и eval. Назовем новую величину change_direction.
5. Добавляем еще 38 фич - фичи предыдущей "свечи" + ее change + ее change_direction.
6. Цель предсказания для каждой свечи - change_direcion следующей свечи.

т.е. брать например, разности между последовательными отсчетами и еще нормировать его на скользящую среднюю

In [298]:
# Параметры
fileName = 'RI.txt'
period = 5
reversePeriod = 20
volumePeriod = 3000
factor = 1
import warnings as wrn
wrn.filterwarnings('ignore')

# Импорты
from sklearn.externals import joblib
import pandas as pd
import numpy as np
import talib as ta
from math import *
import matplotlib.pyplot as plt
import seaborn as sns
from hmmlearn import base
from hmmlearn import hmm
import copy
%matplotlib inline

In [299]:
#читаем таблицу
d = pd.read_csv(fileName)

#разворачиваем таблицу в обратную  сторону
d['i']=range(len(d)-1,-1,-1)
d=d.set_index('i').sort_index(0)
d.head()

,Date,Time,Open,High,Low,Close,Volume
i,,,,,,,
0,10/05/15,1257,78940,78950,78890,78910,170
1,10/05/15,1256,78890,78960,78880,78960,794
2,10/05/15,1255,78870,78900,78860,78880,394
3,10/05/15,1254,78870,78920,78860,78870,631
4,10/05/15,1253,78890,78900,78810,78880,714


In [301]:
d.shape

(1724378, 7)

In [100]:
#считаем обратную скользящую волу с окном reversePeriod

#First, we calculate the periodic return. That's typically a series of daily returns where each return is expressed in continually compounded terms. 
#For each day, we take the natural log of the ratio of stock prices (i.e., price today divided by price yesterday, and so on).
d['r'] = np.log(d.Close/d.Close.shift(1))
d.set_value(0,'r',0)

d['emaR']=d.r.ewm(span = reversePeriod).mean()
d['emaR2']=(d.r*d.r).ewm(span = reversePeriod).mean()
d['reverseSigma'] = np.sqrt(d.emaR2 - d.emaR*d.emaR) * factor
d['emaReverseSigma'] = d.reverseSigma.ewm(span = reversePeriod).mean()

#удаляем временные служебные столбцы

del d['emaR']
del d['emaR2']
del d['reverseSigma']

#разворачиваем таблицу в нормальную сторону
d=d.sort_index(0, ascending=False)

d['i']=range(0,len(d),1)
d=d.set_index('i').sort_index(0)

#считаем отклонение ln(close(t)/close(t-1))-emaReverseSigma
d['ReversePriceMovement'] = d.r-d.emaReverseSigma
del d['r']

In [101]:
d.head()

,Date,Time,Open,High,Low,Close,Volume,emaReverseSigma,ReversePriceMovement
i,,,,,,,,,
0,01/10/06,1031,113000,114000,113000,114000,2,0.002853,-0.005088
1,01/10/06,1033,114260,114260,114255,114255,2,0.002364,-0.026105
2,01/10/06,1035,116000,117000,116000,117000,27,0.001782,0.006802
3,01/10/06,1036,116000,116000,116000,116000,9,0.001634,-0.001634
4,01/10/06,1040,115600,116000,115600,116000,6,0.001579,-0.001579


In [102]:
#считаем прямую скользящую волу с окном period

d['r'] = np.log(d.Close/d.Close.shift(1))
d.set_value(0,'r',0)

d['emaR']=d.r.ewm(span = period).mean()
d['emaR2']=(d.r*d.r).ewm(span = period).mean()
d['sigma'] = np.sqrt(d.emaR2 - d.emaR*d.emaR) * factor
d['emaSigma'] = d.sigma.ewm(span = period).mean()

#удаляем временные служебные столбцы
# del d['r']
del d['emaR']
del d['emaR2']
del d['sigma']

#считаем отклонение ln(close(t)/close(t-1))-emaSigma
d['PriceMovement'] = d.r-d.emaSigma
del d['r']

d.head()

,Date,Time,Open,High,Low,Close,Volume,emaReverseSigma,ReversePriceMovement,emaSigma,PriceMovement
i,,,,,,,,,,,
0,01/10/06,1031,113000,114000,113000,114000,2,0.002853,-0.005088,0.000000,0.000000
1,01/10/06,1033,114260,114260,114255,114255,2,0.002364,-0.026105,0.000657,0.001578
2,01/10/06,1035,116000,117000,116000,117000,27,0.001782,0.006802,0.005657,0.018084
3,01/10/06,1036,116000,116000,116000,116000,9,0.001634,-0.001634,0.008816,-0.017400
4,01/10/06,1040,115600,116000,115600,116000,6,0.001579,-0.001579,0.009479,-0.009479


In [103]:
d.head()

,Date,Time,Open,High,Low,Close,Volume,emaReverseSigma,ReversePriceMovement,emaSigma,PriceMovement
i,,,,,,,,,,,
0,01/10/06,1031,113000,114000,113000,114000,2,0.002853,-0.005088,0.000000,0.000000
1,01/10/06,1033,114260,114260,114255,114255,2,0.002364,-0.026105,0.000657,0.001578
2,01/10/06,1035,116000,117000,116000,117000,27,0.001782,0.006802,0.005657,0.018084
3,01/10/06,1036,116000,116000,116000,116000,9,0.001634,-0.001634,0.008816,-0.017400
4,01/10/06,1040,115600,116000,115600,116000,6,0.001579,-0.001579,0.009479,-0.009479


In [104]:
#считаем дополнительные волатильности по Паркинсону, Гарману-Классу и Роджеру-Сатчелу и осредняем их
#!!!Внимание!!! периоды усреднения для всех прямых волателиьностей равна period. Совершенно не факт, что он должен быть одним для всех. Предмет для подбора!!!
#также возмжно стоит добавить сюда другие волатильности. Ту же ATR например

d['sParkinson'] = 1/(4*log(2))*np.power(np.log(d.High/d.Low),2)
d['sGarmanKlass'] = 0.5*np.power(np.log(d.High/d.Low),2)-(2*log(2)-1)*np.power(d.Close/d.Open,2)
d['sRogerSatchell'] = np.log(d.High/d.Open)*np.log(d.High/d.Open)+np.log(d.Low/d.Open)*np.log(d.Low/d.Close)

d['eParkinson'] = d.sParkinson.ewm(span = period).mean()
d['eGarmanKlass'] = d.sGarmanKlass.ewm(span = period).mean()
d['eRogerSatchell'] = d.sRogerSatchell.ewm(span = period).mean()

d.head()

,Date,Time,Open,High,Low,Close,Volume,emaReverseSigma,ReversePriceMovement,emaSigma,PriceMovement,sParkinson,sGarmanKlass,sRogerSatchell,eParkinson,eGarmanKlass,eRogerSatchell
i,,,,,,,,,,,,,,,,,
0,01/10/06,1031,113000,114000,113000,114000,2,0.002853,-0.005088,0.000000,0.000000,2.799809e-05,-0.393123,0.000078,0.000028,-0.393123,0.000078
1,01/10/06,1033,114260,114260,114255,114255,2,0.002364,-0.026105,0.000657,0.001578,6.906931e-10,-0.386261,0.000000,0.000011,-0.389005,0.000031
2,01/10/06,1035,116000,117000,116000,117000,27,0.001782,0.006802,0.005657,0.018084,2.657468e-05,-0.392946,0.000074,0.000018,-0.390872,0.000051
3,01/10/06,1036,116000,116000,116000,116000,9,0.001634,-0.001634,0.008816,-0.017400,0.000000e+00,-0.386294,0.000000,0.000011,-0.388971,0.000030
4,01/10/06,1040,115600,116000,115600,116000,6,0.001579,-0.001579,0.009479,-0.009479,4.303465e-06,-0.388966,0.000012,0.000008,-0.388969,0.000023


In [105]:
d['i']=range(0,len(d),1)
d=d.set_index('i').sort_index(0)
d.head()

,Date,Time,Open,High,Low,Close,Volume,emaReverseSigma,ReversePriceMovement,emaSigma,PriceMovement,sParkinson,sGarmanKlass,sRogerSatchell,eParkinson,eGarmanKlass,eRogerSatchell
i,,,,,,,,,,,,,,,,,
0,01/10/06,1031,113000,114000,113000,114000,2,0.002853,-0.005088,0.000000,0.000000,2.799809e-05,-0.393123,0.000078,0.000028,-0.393123,0.000078
1,01/10/06,1033,114260,114260,114255,114255,2,0.002364,-0.026105,0.000657,0.001578,6.906931e-10,-0.386261,0.000000,0.000011,-0.389005,0.000031
2,01/10/06,1035,116000,117000,116000,117000,27,0.001782,0.006802,0.005657,0.018084,2.657468e-05,-0.392946,0.000074,0.000018,-0.390872,0.000051
3,01/10/06,1036,116000,116000,116000,116000,9,0.001634,-0.001634,0.008816,-0.017400,0.000000e+00,-0.386294,0.000000,0.000011,-0.388971,0.000030
4,01/10/06,1040,115600,116000,115600,116000,6,0.001579,-0.001579,0.009479,-0.009479,4.303465e-06,-0.388966,0.000012,0.000008,-0.388969,0.000023


In [131]:
data_frame_X = d[['Date','Time', 'emaSigma', 'sParkinson','sGarmanKlass','sRogerSatchell','eParkinson','eGarmanKlass','eRogerSatchell']]
data_frame_Y= d[['emaReverseSigma']]
df_X = d.loc[(d.index < 100000),['emaSigma','emaReverseSigma']]
df_Y = d.loc[(d.index < 100000),['emaReverseSigma']]

In [132]:
from sklearn.model_selection import train_test_split

def create_GMMHMM_models(data_frame_X, data_frame_Y, n_hidden_st):
    
    #переделываем data frame в numpy array
    X = np.array(data_frame_X.values)
    Y = np.array(data_frame_Y.values)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.001, random_state=42)
    remodel = hmm.GMMHMM(n_components=n_hidden_st, covariance_type="diag", n_iter=100)
    remodel.fit(X_train)

In [237]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.001, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(99900, 2) (100, 2) (99900, 1) (100, 1)


In [238]:
remodel = hmm.GaussianHMM(n_components=4,covariance_type="diag",random_state=42)

In [239]:
#X = np.array(X_train.values)
remodel.fit(X_train)

GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=4, n_iter=10, params='stmc',
      random_state=42, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)

In [289]:
print('Transition matrix for model\n',remodel.transmat_)
print('\nlogprobability for X_TRAIN  ',remodel.score(X_train))
print('\nlogprobability for X_test  ',remodel.score(X_test)) 
print('\nScore samples X_train  \n',remodel.score_samples(X_train))

Transition matrix for model
 [[ 0.55395306  0.10327054  0.32547582  0.01730059]
 [ 0.55528114  0.10287205  0.32594082  0.015906  ]
 [ 0.55396619  0.10337079  0.32552019  0.01714282]
 [ 0.56298043  0.09678293  0.32654355  0.01369308]]

logprobability for X_TRAIN   1227053.11381

logprobability for X_test   1230.6509154

Score samples X_train  
 (1227053.1138080743, array([[  7.91624688e-01,   3.35347658e-03,   2.05021830e-01,
          4.77810044e-09],
       [  7.25553208e-01,   1.82803016e-02,   2.55743307e-01,
          4.23183133e-04],
       [  7.28481427e-01,   1.79661549e-02,   2.53154048e-01,
          3.98370733e-04],
       ..., 
       [  7.31502359e-01,   1.74931756e-02,   2.50614061e-01,
          3.90404192e-04],
       [  7.31279775e-01,   1.74748253e-02,   2.50849994e-01,
          3.95406418e-04],
       [  7.12998748e-01,   2.05412700e-02,   2.66010970e-01,
          4.49012071e-04]]))


In [241]:
t = 30
Z1 = remodel.predict(X_train)
logprobofseq,Z2 = remodel.decode(X_train)
print(Z1[:t])
print(Z2[:t])
print(logprobofseq)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 2]
1189050.2881786015


In [242]:
#Compute the posterior probability for each state in the model

posteriors=remodel.predict_proba(X_train) #State-membership probabilities for each sample from X.

print(posteriors.shape)
for i in range(t-20):
    print(posteriors[i],Z1[i])

(99900, 4)
[  7.91624688e-01   3.35347658e-03   2.05021830e-01   4.77810044e-09] 0
[  7.25553208e-01   1.82803016e-02   2.55743307e-01   4.23183133e-04] 0
[  7.28481427e-01   1.79661549e-02   2.53154048e-01   3.98370733e-04] 0
[  7.25496953e-01   1.81413896e-02   2.55920963e-01   4.40694392e-04] 0
[  6.79146135e-01   2.71051379e-02   2.93185558e-01   5.63169481e-04] 0
[  7.31472370e-01   1.74967595e-02   2.50641624e-01   3.89246093e-04] 0
[  6.60922328e-01   3.08301946e-02   3.07599095e-01   6.48383174e-04] 0
[  6.77276589e-01   2.74012572e-02   2.94742839e-01   5.79313793e-04] 0
[ 0.58651207  0.04946289  0.36283672  0.00118832] 0
[  7.28271971e-01   1.78332688e-02   2.53475344e-01   4.19416467e-04] 0


In [293]:
lgprob, hiddenst = remodel.decode(X_test)
print('Logprob of seq by viterbi on TEST \n\n',lgprob, '\n\n seq for X_test by viterbi \n\n',hiddenst)

Logprob of seq by viterbi on TEST 

 1192.3766459526753 

 seq for X_test by viterbi 

 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0
 0 0 0 0 0 0 1 0 0 0 0 2 0 1 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]


In [248]:
generate_sample, hst_seq = remodel.sample(100,random_state=6)
posteriorsTEST = remodel.predict_proba(X_test)
print('Вероятность каждого скрытого состояния для последнего семпла в тестовом куске - 99ый элемент',posteriorsTEST[-1])
print('\nСгенерированная выборка из модели\n',generate_sample, '\nИ последовательность скрытых состояний', hst_seq)
posterGENERATED = remodel.predict_proba(generate_sample)
print('\nДля которых вероятности каждого скрытого состояния слелующие\n',posterGENERATED)

Вероятность каждого скрытого состояния для последнего семпла в тестовом куске - 99ый элемент [  7.22594044e-01   1.85704230e-02   2.58383343e-01   4.52189512e-04]

Сгенерированная выборка из модели
 [[  1.36658585e-03   4.40959440e-04]
 [  7.18038997e-04   2.31362551e-04]
 [ -3.54228452e-04   2.38786821e-04]
 [  3.36123517e-03   8.41812966e-04]
 [  1.58679458e-03   1.29254032e-03]
 [  2.00914193e-03   6.56104126e-03]
 [  1.27867135e-03   1.27446312e-03]
 [  9.18984556e-04   2.12055030e-03]
 [  5.45168010e-04   1.01471855e-03]
 [  1.04120706e-03   7.66510091e-05]
 [  1.58160051e-03   1.59853586e-03]
 [  2.89638971e-04  -9.37258913e-05]
 [  2.22581388e-04   1.94421883e-03]
 [ -4.08216329e-03   2.75836181e-04]
 [  5.47967872e-04   2.75298221e-04]
 [  1.37561474e-04   1.50496271e-03]
 [ -4.94779001e-04   5.32966018e-04]
 [  2.88124026e-04   7.61284121e-05]
 [  1.29073729e-03   1.01041768e-03]
 [  4.16162148e-04   1.10324719e-03]
 [ -1.99978164e-03   5.93129112e-04]
 [  1.03846204e-03   1.4

In [295]:
#y_test = np.array(y_test.values)
Y_move = np.zeros((100))
for i in range(len(y_test)-1) :
    if (y_test[i+1] - y_test[i])>0:
        Y_move[i] = 1
    else: 
        Y_move[i] = -1
print('\n emaReverse movement in future on y_test \n',Y_move)


 emaReverse move in future on y-test 
 [-1.  1.  1. -1.  1. -1.  1. -1.  1. -1.  1. -1. -1.  1. -1. -1.  1.  1.
  1. -1.  1. -1. -1.  1. -1. -1. -1. -1.  1.  1. -1.  1.  1. -1.  1. -1.
 -1.  1. -1. -1.  1.  1. -1. -1. -1.  1. -1.  1. -1.  1. -1.  1. -1.  1.
  1. -1.  1.  1. -1. -1.  1.  1. -1.  1. -1.  1.  1.  1. -1.  1.  1. -1.
  1.  1.  1. -1. -1. -1.  1. -1. -1.  1. -1. -1. -1.  1.  1.  1. -1.  1.
 -1.  1.  1. -1.  1.  1.  1. -1. -1.  0.]


In [296]:
Y_predict = generate_sample[:,1]
Y_pred_move = np.zeros((100))
for i in range(len(Y_predict)-1) :
    if (Y_predict[i+1] - Y_predict[i])>0:
        Y_pred_move[i] = 1
    else: 
        Y_pred_move[i] = -1
print('\n emaReverse movement in future from generated samples \n',Y_pred_move)


 emaReverse movement in future from generated samples 
 [-1.  1.  1.  1.  1. -1.  1. -1. -1.  1. -1.  1. -1. -1.  1. -1. -1.  1.
  1. -1.  1. -1.  1. -1.  1. -1. -1.  1.  1. -1. -1.  1. -1.  1.  1. -1.
 -1.  1. -1.  1.  1.  1. -1.  1. -1. -1.  1.  1. -1.  1. -1. -1.  1.  1.
  1.  1. -1.  1. -1. -1. -1. -1.  1.  1.  1. -1.  1. -1. -1.  1.  1. -1.
  1. -1. -1.  1.  1.  1. -1.  1.  1.  1. -1. -1.  1.  1. -1.  1. -1. -1.
  1. -1.  1. -1.  1. -1.  1. -1.  1.  0.]


In [284]:
# metrics
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(Y_move, Y_pred_move))

             precision    recall  f1-score   support

       -1.0       0.55      0.53      0.54        49
        0.0       1.00      1.00      1.00         1
        1.0       0.56      0.58      0.57        50

avg / total       0.56      0.56      0.56       100

